In [9]:
import pandas as pd
from raceplotly.plots import barplot

In [10]:
athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['result_id','athlete','athlete_id','pos'])


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1

athlete_results.drop(repeat, inplace=True)
athlete_results.drop(columns=['edition_id', 'isTeamSport', 'index'], inplace=True)
athlete_results['edition'] = athlete_results['edition'].str.split().str[0].astype(int)

athlete_results["medal"] = athlete_results["medal"].notnull().mul(1)

display(athlete_results)

,edition,country_noc,sport,event,medal
0,1908,ANZ,Athletics,"100 metres, Men",0
1,1908,ANZ,Athletics,"400 metres, Men",0
2,1908,ANZ,Athletics,"800 metres, Men",0
3,1908,ANZ,Athletics,"800 metres, Men",0
4,1908,ANZ,Athletics,"800 metres, Men",0
...,...,...,...,...,...
253177,2020,USA,Wrestling,"Featherweight, Freestyle, Women",0
253178,2020,USA,Wrestling,"Lightweight, Freestyle, Women",1
253179,2020,USA,Wrestling,"Middleweight, Freestyle, Women",0
253180,2020,USA,Wrestling,"Light-Heavyweight, Freestyle, Women",1


In [11]:
sport = "Athletics"

athlete_results = athlete_results[athlete_Event_Results['sport'] == sport]

df = pd.DataFrame(athlete_results.groupby(['edition','country_noc']).sum()).reset_index()

for i in df['edition'].unique():
    for j in df['country_noc'].unique():
        if df[(df['edition'] == i) & (df['country_noc'] == j)].empty:
            df.loc[len(df), ['edition','country_noc', 'medal']] = i, j, 0

df

C:\Users\pcost\AppData\Local\Temp\ipykernel_21012\3542845143.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,edition,country_noc,medal
0,1896.0,BUL,0.0
1,1896.0,FRA,0.0
2,1896.0,GRE,12.0
3,1900.0,BOH,1.0
4,1900.0,CUB,2.0
...,...,...,...
5575,2020.0,CPV,0.0
5576,2020.0,MNE,0.0
5577,2020.0,SVK,0.0
5578,2020.0,SRB,0.0


In [12]:
df['edition'] = df['edition'].astype(int)
df['medal'] = df['medal'].astype(int)

In [13]:
new_medal = []
for i in range(len(df)):
    edition = df.loc[i, 'edition']
    country = df.loc[i, 'country_noc']
    medals = df.loc[i, 'medal']

    new_medal.append(df[(df['edition'] <= edition) & (df['country_noc'] == country)]['medal'].sum())

df['medals'] = new_medal
df.drop(columns=['medal'], inplace=True)


df

,edition,country_noc,medals
0,1896,BUL,0
1,1896,FRA,0
2,1896,GRE,12
3,1900,BOH,1
4,1900,CUB,2
...,...,...,...
5575,2020,CPV,0
5576,2020,MNE,0
5577,2020,SVK,4
5578,2020,SRB,3


In [19]:

#1 Assign color to each unique item
from random import sample
import plotly.graph_objects as go

colors = {item: 'rgb({}, {}, {})'.format(*sample(range(256), 3)) for item in df['country_noc'].unique()}


my_raceplot = barplot(df, item_column='country_noc', value_column='medals', time_column='edition')

my_raceplot.plot(item_label = 'Top 10 countries', value_label = 'Amount of Medals', time_label='Year ', frame_duration = 800)
